In [17]:
import tweepy
import json
from pymongo import MongoClient
from collections import Counter
from collections.abc import MutableMapping
import logging
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import community
import seaborn as sns
from fastprogress import master_bar, progress_bar
import time
import re

#Logger
logging.basicConfig(filename='Heatmap2.log', format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p', level=logging.INFO)
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logging.getLogger().addHandler(ch)

In [ ]:
########################
#          CHES         #
########################

CHES_ideological_position ={}

CHES_ideological_position['VOX'] = [9.71]
CHES_ideological_position['PP'] = [8.06]
CHES_ideological_position['CS'] = [7.20]
CHES_ideological_position['PDECAT'] = [6.66]
CHES_ideological_position['PSOE'] = [3.60]
CHES_ideological_position['ERC'] = [3.20]
CHES_ideological_position['MAS_PAIS'] = [2.73]
CHES_ideological_position['PODEMOS'] = [1.93]
CHES_ideological_position['IU'] = [1.86]

print(CHES_ideological_position)

In [ ]:
########################
# PREPARE HEATMAP DATA #
########################

COMMUNITIES_LIST = ["VOX", "PP", "CS", "PDECAT", "PSOE", "ERC", "MAS_PAIS", "PODEMOS", "IU"]


matrix = np.zeros((len(COMMUNITIES_LIST),len(COMMUNITIES_LIST)))

for A in COMMUNITIES_LIST:
    for B in COMMUNITIES_LIST:
        a = np.array(CHES_ideological_position[A])
        b = np.array(CHES_ideological_position[B])
        distance = np.linalg.norm(a-b)
        #https://en.wikipedia.org/wiki/Euclidean_distance
                
        matrix[COMMUNITIES_LIST.index(A)][COMMUNITIES_LIST.index(B)] = distance 
        
print(matrix)  

In [ ]:
################
# PLOT HEATMAP #
################

def plot_heatmap(m, x_values, y_values, title, xlabel, ylabel, save_fig=False, label_rotation=None):
    """
        Creates a heatmap image from a numpy matrix.

    :param m: 2-dimensional numpy matrix with values to plot
    :param x_values: list of strings for xticks
    :param y_values: list of strings for yticks
    :param title: string, title of the plot
    :param xlabel: string, label of the x axis
    :param ylabel: string, label of the y axis
    :param save_fig: False / "show" / figname , do not show imatge / show it inline / write it to pdf (figure name)
    :param label_rotation: None / int, whether to rotate x ticks (degrees)
    :return:
    """
    
    fig = plt.figure(figsize=(20, 20))
    ax = fig.add_subplot(1, 1, 1)

    im = ax.imshow(m.transpose(), origin='lower', cmap='Reds', alpha=0.7, aspect='auto')
    #im = ax.imshow(m.transpose(), origin='lower', cmap='jet', alpha=0.7)

    # Colorbar
    cbar = ax.figure.colorbar(im, ax=ax)

    # Loop over data dimensions and create text annotations.
    mt = m.transpose()
    for i in range(len(x_values)):
        for j in range(len(y_values)):
            text = ax.text(j, i, "{:.2f}".format(mt[i, j]), ha="center", va="center", color="black", alpha=1, fontsize=18)

    # Named ticks
    ax.set_xticks(np.arange(len(x_values)))
    ax.set_yticks(np.arange(len(y_values)))
    ax.set_xticklabels(x_values)
    ax.set_yticklabels(y_values)

    # Axis labels
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)

    # Rotate x labels
    if label_rotation:
        plt.xticks(rotation=label_rotation)

    fig.tight_layout()
    plt.title(title)
    
    plt.rcParams.update({'font.size': 24})

    # Output result
    if not save_fig:
        pass
    elif save_fig == "show":
        plt.show()
    else:
        plt.savefig(save_fig + '.pdf', format='pdf', dpi=600)
        plt.close()

        
######INPUT DATA#####

m = matrix
x_values = COMMUNITIES_LIST
y_values = COMMUNITIES_LIST
title = "Ideological Distance"
xlabel = "Passive"
ylabel = "Active"
save_fig = "show"

plot_heatmap(m, x_values, y_values, title, xlabel, ylabel, save_fig, 60)

In [ ]:
################################
# PLOT HEATMAP (NO REDUNDANCY) #
################################

COMMUNITIES_LIST = ["VOX", "PP", "CS", "PDCAT", "PSOE", "ERC", "MP", "PS", "IU"]

matrix_df = pd.DataFrame(data=matrix,    # values
              index=COMMUNITIES_LIST,    # 1st column as index
              columns=COMMUNITIES_LIST)  # 1st row as the column names

# Calculate pairwise-correlation
m2 = matrix_df.corr()

# Create a mask
mask = np.triu(np.ones_like(m2, dtype=bool))

np.fill_diagonal(mask, False)

# Create a custom divergin palette
cmap = sns.diverging_palette(250, 15, s=75, l=40,
                            n=9, center="light", as_cmap=True)

plt.figure(figsize=(16, 12))
sns.heatmap(matrix_df, mask=mask, center=0, annot=True,
            fmt='.2f', square=True, cmap=cmap)

plt.title("Ideological Distance (CHES)")

#plt.savefig("HEATMAP-CHES.png")
plt.show()

In [ ]:
#############################################
#          MANIFESTO RESEARCH GROUP         #
#############################################

Manifesto_ideological_position ={}

Manifesto_ideological_position['VOX'] = [8.30]
Manifesto_ideological_position['PP'] = [5.45]
Manifesto_ideological_position['CUP'] = [4.11]
Manifesto_ideological_position['CS'] = [3.47]
Manifesto_ideological_position['JXCAT'] = [3.31]
Manifesto_ideological_position['ERC'] = [2.26]
Manifesto_ideological_position['PSOE'] = [2.13]
Manifesto_ideological_position['MAS_PAIS'] = [1.99]
Manifesto_ideological_position['PODEMOS'] = [1.24]
Manifesto_ideological_position['IU'] = [1.03]

print(Manifesto_ideological_position)

In [ ]:
########################
# PREPARE HEATMAP DATA #
########################


COMMUNITIES_LIST = ["VOX", "PP", "CUP", "CS", "JXCAT", "ERC", "PSOE", "MAS_PAIS", "PODEMOS", "IU"]


matrix = np.zeros((len(COMMUNITIES_LIST),len(COMMUNITIES_LIST)))

for A in COMMUNITIES_LIST:
    for B in COMMUNITIES_LIST:
        a = np.array(Manifesto_ideological_position[A])
        b = np.array(Manifesto_ideological_position[B])
        distance = np.linalg.norm(a-b)
        #https://en.wikipedia.org/wiki/Euclidean_distance
                
        matrix[COMMUNITIES_LIST.index(A)][COMMUNITIES_LIST.index(B)] = distance #TO:DO: Revise SCALING!!
        
print(matrix)  

In [ ]:
################################
# PLOT HEATMAP (NO REDUNDANCY) #
################################

COMMUNITIES_LIST = ["VOX", "PP", "CUP", "CS", "JXC", "ERC", "PSOE", "MP", "PS", "IU"]

matrix_df = pd.DataFrame(data=matrix,    # values
              index=COMMUNITIES_LIST,    # 1st column as index
              columns=COMMUNITIES_LIST)  # 1st row as the column names

# Calculate pairwise-correlation
m2 = matrix_df.corr()

# Create a mask
mask = np.triu(np.ones_like(m2, dtype=bool))

np.fill_diagonal(mask, False)

# Create a custom divergin palette
cmap = sns.diverging_palette(250, 15, s=75, l=40,
                            n=9, center="light", as_cmap=True)

plt.figure(figsize=(16, 12))
sns.heatmap(matrix_df, mask=mask, center=0, annot=True,
            fmt='.2f', square=True, cmap=cmap)

plt.title("Ideological Distance (MANIFESTO)")

#plt.savefig("HEATMAP-MANIFESTO.png")
plt.show()

In [ ]:
###########################################
#          REGIONAL MANIFESTO CAT         #
###########################################

CEO_ideological_position ={}

CEO_ideological_position['CiudadanosCs'] = [6.04, 4.39]
CEO_ideological_position['JuntsXCat'] = [4.94, 6.29]
CEO_ideological_position['Esquerra_ERC'] = [2.94, 7.11]
CEO_ideological_position['socialistes_cat'] = [5.31, 5.36]
CEO_ideological_position['EnComu_Podem'] = [2.70, 5.78]
CEO_ideological_position['cupnacional'] = [1.51, 6.84]
CEO_ideological_position['populares'] = [9.57, 4.67]
CEO_ideological_position['vox_es'] = [8.33, 1.32]

print(CEO_ideological_position)

In [ ]:
########################
# PREPARE HEATMAP DATA #
########################


COMMUNITIES_LIST = ["VOX", "PP", "CS", "PSC", "COMUNS", "ERC", "CUP", "JXCAT"]
PARTY_NAMES_LIST = ['vox_es', 'populares', 'CiudadanosCs', 'socialistes_cat',
                    'EnComu_Podem','Esquerra_ERC', 'cupnacional', 'JuntsXCat']


matrix = np.zeros((len(COMMUNITIES_LIST),len(COMMUNITIES_LIST)))

for A in COMMUNITIES_LIST:
    for B in COMMUNITIES_LIST:
        a = np.array(CEO_ideological_position[PARTY_NAMES_LIST[COMMUNITIES_LIST.index(A)]])
        b = np.array(CEO_ideological_position[PARTY_NAMES_LIST[COMMUNITIES_LIST.index(B)]])
        distance = np.linalg.norm(a-b)
        #https://en.wikipedia.org/wiki/Euclidean_distance
                
        matrix[COMMUNITIES_LIST.index(A)][COMMUNITIES_LIST.index(B)] = distance #TO:DO: Revise SCALING!!
        
print(matrix)   

In [ ]:
################
# PLOT HEATMAP #
################

def plot_heatmap(m, x_values, y_values, title, xlabel, ylabel, save_fig=False, label_rotation=None):
    """
        Creates a heatmap image from a numpy matrix.

    :param m: 2-dimensional numpy matrix with values to plot
    :param x_values: list of strings for xticks
    :param y_values: list of strings for yticks
    :param title: string, title of the plot
    :param xlabel: string, label of the x axis
    :param ylabel: string, label of the y axis
    :param save_fig: False / "show" / figname , do not show imatge / show it inline / write it to pdf (figure name)
    :param label_rotation: None / int, whether to rotate x ticks (degrees)
    :return:
    """
    
    fig = plt.figure(figsize=(20, 20))
    ax = fig.add_subplot(1, 1, 1)

    im = ax.imshow(m.transpose(), origin='lower', cmap='Reds', alpha=0.7, aspect='auto')
    #im = ax.imshow(m.transpose(), origin='lower', cmap='jet', alpha=0.7)

    # Colorbar
    cbar = ax.figure.colorbar(im, ax=ax)

    # Loop over data dimensions and create text annotations.
    mt = m.transpose()
    for i in range(len(x_values)):
        for j in range(len(y_values)):
            text = ax.text(j, i, "{:.2f}".format(mt[i, j]), ha="center", va="center", color="black", alpha=1, fontsize=18)

    # Named ticks
    ax.set_xticks(np.arange(len(x_values)))
    ax.set_yticks(np.arange(len(y_values)))
    ax.set_xticklabels(x_values)
    ax.set_yticklabels(y_values)

    # Axis labels
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)

    # Rotate x labels
    if label_rotation:
        plt.xticks(rotation=label_rotation)

    fig.tight_layout()
    plt.title(title)
    
    plt.rcParams.update({'font.size': 24})

    # Output result
    if not save_fig:
        pass
    elif save_fig == "show":
        plt.show()
    else:
        plt.savefig(save_fig + '.pdf', format='pdf', dpi=600)
        plt.close()

        
######INPUT DATA#####

m = matrix
x_values = COMMUNITIES_LIST
y_values = COMMUNITIES_LIST
title = "Ideological Distance (CEO)"
xlabel = "Passive"
ylabel = "Active"
save_fig = "show"

plot_heatmap(m, x_values, y_values, title, xlabel, ylabel, save_fig, 60)

In [ ]:
################################
# PLOT HEATMAP (NO REDUNDANCY) #
################################

matrix_df = pd.DataFrame(data=matrix,    # values
              index=COMMUNITIES_LIST,    # 1st column as index
              columns=COMMUNITIES_LIST)  # 1st row as the column names

# Calculate pairwise-correlation
m2 = matrix_df.corr()

# Create a mask
mask = np.triu(np.ones_like(m2, dtype=bool))

np.fill_diagonal(mask, False)

# Create a custom divergin palette
cmap = sns.diverging_palette(250, 15, s=75, l=40,
                            n=9, center="light", as_cmap=True)

plt.figure(figsize=(16, 12))
sns.heatmap(matrix_df, mask=mask, center=0, annot=True,
            fmt='.2f', square=True, cmap=cmap)

plt.title("Ideological Distance (MANIFESTO-CAT)")

#plt.savefig("CAT-HEATMAP-MANIFESTO.png")
plt.show()

In [ ]:
###########################################
#          REGIONAL MANIFESTO MAD         #
###########################################

CIS_ideological_position ={}

CIS_ideological_position['vox_es'] = [7.66]
CIS_ideological_position['populares'] = [6.68]
CIS_ideological_position['CiudadanosCs'] = [5.79]
CIS_ideological_position['PSOE'] = [4.27]
CIS_ideological_position['MasMadrid__'] = [3.34]
CIS_ideological_position['PODEMOS'] = [1.37]


print(CIS_ideological_position)

In [ ]:
########################
# PREPARE HEATMAP DATA #
########################


COMMUNITIES_LIST = ["VOX", "PP", "CS", "PSOE",  "MAS_MAD", "PODEMOS"]
PARTY_NAMES_LIST = ['vox_es', 'populares', 'CiudadanosCs', 'PSOE', 'MasMadrid__', 'PODEMOS']


matrix = np.zeros((len(COMMUNITIES_LIST),len(COMMUNITIES_LIST)))

for A in COMMUNITIES_LIST:
    for B in COMMUNITIES_LIST:
        a = np.array(CIS_ideological_position[PARTY_NAMES_LIST[COMMUNITIES_LIST.index(A)]])
        b = np.array(CIS_ideological_position[PARTY_NAMES_LIST[COMMUNITIES_LIST.index(B)]])
        distance = np.linalg.norm(a-b)
        #https://en.wikipedia.org/wiki/Euclidean_distance
                
        matrix[COMMUNITIES_LIST.index(A)][COMMUNITIES_LIST.index(B)] = distance #TO:DO: Revise SCALING!!
        
print(matrix)        

In [ ]:
################################
# PLOT HEATMAP (NO REDUNDANCY) #
################################

COMMUNITIES_LIST = ["VOX", "PP", "CS", "PSOE",  "MM", "PS"]

matrix_df = pd.DataFrame(data=matrix,    # values
              index=COMMUNITIES_LIST,    # 1st column as index
              columns=COMMUNITIES_LIST)  # 1st row as the column names

# Calculate pairwise-correlation
m2 = matrix_df.corr()

# Create a mask
mask = np.triu(np.ones_like(m2, dtype=bool))

np.fill_diagonal(mask, False)

# Create a custom divergin palette
cmap = sns.diverging_palette(250, 15, s=75, l=40,
                            n=9, center="light", as_cmap=True)

plt.figure(figsize=(16, 12))
sns.heatmap(matrix_df, mask=mask, center=0, annot=True,
            fmt='.2f', square=True, cmap=cmap)

plt.title("Ideological Distance (MANIFESTO-MAD)")

#plt.savefig("MAD-HEATMAP-MANIFESTO.png")
plt.show()

In [ ]:
###########################################
#          REGIONAL MANIFESTO AND         #
###########################################

CES_ideological_position ={}

CES_ideological_position['vox_es'] = [7.01]
CES_ideological_position['populares'] = [5.79]
CES_ideological_position['CiudadanosCs'] = [5.98]
CES_ideological_position['PSOE'] = [4.34]
CES_ideological_position['PODEMOS'] = [2.20]
CES_ideological_position['AdelanteAND'] = [2.20]


print(CES_ideological_position)

In [ ]:
########################
# PREPARE HEATMAP DATA #
########################


COMMUNITIES_LIST = ["VOX", "PP", "CS", "PSOE", "POR_AND", "ADELANTE_AND"]
PARTY_NAMES_LIST = ['vox_es', 'populares','CiudadanosCs', 'PSOE', 'PODEMOS', 'AdelanteAND']


matrix = np.zeros((len(COMMUNITIES_LIST),len(COMMUNITIES_LIST)))

for A in COMMUNITIES_LIST:
    for B in COMMUNITIES_LIST:
        a = np.array(CES_ideological_position[PARTY_NAMES_LIST[COMMUNITIES_LIST.index(A)]])
        b = np.array(CES_ideological_position[PARTY_NAMES_LIST[COMMUNITIES_LIST.index(B)]])
        distance = np.linalg.norm(a-b)
        #https://en.wikipedia.org/wiki/Euclidean_distance
                
        matrix[COMMUNITIES_LIST.index(A)][COMMUNITIES_LIST.index(B)] = distance #TO:DO: Revise SCALING!!
        
print(matrix)  

In [ ]:
################################
# PLOT HEATMAP (NO REDUNDANCY) #
################################

COMMUNITIES_LIST = ["VOX", "PP", "CS", "PSOE", "PA", "AA"]

matrix_df = pd.DataFrame(data=matrix,    # values
              index=COMMUNITIES_LIST,    # 1st column as index
              columns=COMMUNITIES_LIST)  # 1st row as the column names

# Calculate pairwise-correlation
m2 = matrix_df.corr()

# Create a mask
mask = np.triu(np.ones_like(m2, dtype=bool))

np.fill_diagonal(mask, False)

# Create a custom divergin palette
cmap = sns.diverging_palette(250, 15, s=75, l=40,
                            n=9, center="light", as_cmap=True)

plt.figure(figsize=(16, 12))
sns.heatmap(matrix_df, mask=mask, center=0, annot=True,
            fmt='.2f', square=True, cmap=cmap)

plt.title("Ideological Distance (MANIFESTO-AND)")

#plt.savefig("AND-HEATMAP-MANIFESTO.png")
plt.show()